In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers
from sklearn.ensemble import RandomForestRegressor
class Mics_Model:
    def __init__(self, dataset_dir, use_encoder=True, sampling_method="Vanilla", global_model="NN"):
        self.dataset_dir = dataset_dir
        self.use_encoder = use_encoder
        self.sampling_method = sampling_method
        self.global_model = global_model
        self.raw_data = None
            
    #ASSUMPTION: label column is in the end and all other columns are features. There is no index column in the data.
    def get_raw_data(self):
        raw_data = pd.read_csv(self.dataset_dir, index_col=None)
        raw_data = raw_data.fillna(df.mean())
        self.raw_data = raw_data
                
    def get_vanilla_encoder_model(self, inp_size):
        inputs = keras.layers.Input(shape=(inp_size))
        h1 = keras.layers.Dense(10, activation="relu")(inputs)
        h1 = keras.layers.Dense(10, activation="relu")(inputs)        
        outputs = keras.layers.Dense(inp_size, activation="relu")(h1)
        return keras.Model(inputs,outputs)
    
    class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon #multiplies with std
    
    def get_variatonal_encoder_model(self, inp_size):
        inputs = layers.Input(shape=(inp_size))
        h1 = layers.Dense(10, activation="relu")(inputs)
        z_mean = layers.Dense(inp_size, name="z_mean")(h1)
        z_log_var = layers.Dense(inp_size, name="z_log_var")(h1)
        outputs = Sampling()([z_mean, z_log_var])
        return keras.Model(inputs,outputs)
    #Here, new sampling methods can be added here 
    
    def get_nn_model(self, inp_sizes, drop_out=0.25, hidden_num = 4, hidden_size=32):
        inp_group_count = len(inp_sizes)
        inputs = [None]*inp_group_count
        for i in range(inp_group_count):
            inputs[i] = keras.layers.Input(shape=(inp_sizes[i]), name="input_"+str(i))
        if self.use_encoder == True:
            encoders = [None]*inp_group_count
            if self.sampling_method == "Vanilla":
                for j in range(inp_group_count):
                    encoders[j] = self.get_vanilla_encoder_model(inp_sizes[j])
            elif self.sampling_method == "Variational":
                for j in range(inp_group_count):
                    encoders[j] = self.get_variatonal_encoder_model(inp_sizes[j])
            #This place can be extended if new sampling methods are added.
            global_inputs = [None]*inp_group_count
            for k in range(inp_group_count):
                global_inputs[k] = encoders[k](inputs[k])
                global_input = keras.layers.concatenate(global_inputs)
        else:
            global_input = keras.layers.concatenate(inputs)
            
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(global_inp)
        h = keras.layers.Dropout(drop_out)(h)
        for hidden in range(hidden_num):
            h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
            h = keras.layers.Dropout(drop_out)(h) 

        outputs = keras.layers.Dense(1, activation="relu")(h)    
        return keras.Model(inputs=[inputs_A, inputs_B, inputs_C], outputs = outputs) 
                


In [2]:
def var_lis(lis):
    for i in range(len(lis)):
        print(lis[i])
var_lis([1,4,6,8])
              

1
4
6
8


In [4]:
a = [None]*8
b = [3,7,9]
for i in range(len(b)):
    a[i] = b[i]
a    

[3, 7, 9, None, None, None, None, None]